In [7]:
import sys
sys.path.append('fairseq/')
import torch
from examples.data2vec.models.data2vec2 import Data2VecMultiModel
from examples.data2vec.models.data2vec2 import Data2VecMultiConfig

In [8]:
state_dict = torch.load('nlp_base.pt')

In [10]:
state_dict.keys()

dict_keys(['args', 'cfg', 'model', 'criterion', 'optimizer_history', 'task_state', 'extra_state', 'last_optimizer_state'])

In [13]:
state_dict['cfg']

{'_name': None,
 'common': {'_name': None,
  'no_progress_bar': False,
  'log_interval': 200,
  'log_format': 'json',
  'log_file': None,
  'aim_repo': None,
  'aim_run_hash': None,
  'tensorboard_logdir': 'tb',
  'wandb_project': None,
  'azureml_logging': False,
  'seed': 1,
  'cpu': False,
  'tpu': False,
  'bf16': False,
  'memory_efficient_bf16': False,
  'fp16': True,
  'memory_efficient_fp16': False,
  'fp16_no_flatten_grads': True,
  'fp16_init_scale': 128,
  'fp16_scale_window': None,
  'fp16_scale_tolerance': 0.0,
  'on_cpu_convert_precision': False,
  'min_loss_scale': 0.0001,
  'threshold_loss_scale': None,
  'amp': False,
  'amp_batch_retries': 2,
  'amp_init_scale': 128,
  'amp_scale_window': None,
  'user_dir': '/data/home/wnhsu/fairseq_repos/fairseq-py-d2v-dev/examples/data2vec',
  'empty_cache_freq': 0,
  'all_gather_list_size': 16384,
  'model_parallel_size': 1,
  'quantization_config_path': None,
  'profile': False,
  'reset_logging': False,
  'suppress_crashes': Fal

In [15]:
state_dict['model'].keys()

odict_keys(['modality_encoders.TEXT.local_encoder.embed_tokens.weight', 'modality_encoders.TEXT.local_encoder.embed_positions.weight', 'modality_encoders.TEXT.local_encoder.layernorm_embedding.weight', 'modality_encoders.TEXT.local_encoder.layernorm_embedding.bias', 'modality_encoders.TEXT.decoder.blocks.0.0.weight', 'modality_encoders.TEXT.decoder.blocks.0.0.bias', 'modality_encoders.TEXT.decoder.blocks.1.0.weight', 'modality_encoders.TEXT.decoder.blocks.1.0.bias', 'modality_encoders.TEXT.decoder.blocks.2.0.weight', 'modality_encoders.TEXT.decoder.blocks.2.0.bias', 'modality_encoders.TEXT.decoder.blocks.3.0.weight', 'modality_encoders.TEXT.decoder.blocks.3.0.bias', 'modality_encoders.TEXT.decoder.blocks.4.0.weight', 'modality_encoders.TEXT.decoder.blocks.4.0.bias', 'modality_encoders.TEXT.decoder.proj.0.weight', 'modality_encoders.TEXT.decoder.proj.0.bias', 'modality_encoders.TEXT.decoder.proj.2.weight', 'modality_encoders.TEXT.decoder.proj.2.bias', 'blocks.0.norm1.weight', 'blocks.0.

In [24]:
def count_parameters_from_state_dict(state_dict):
    total_params = 0
    
    def count_parameters(obj):
        nonlocal total_params
        if isinstance(obj, torch.Tensor):
            total_params += torch.numel(obj)
        elif isinstance(obj, dict):
            for k in obj:
                count_parameters(obj[k])
    
    count_parameters(state_dict)
    return total_params

In [25]:
count_parameters_from_state_dict(state_dict['model'])

238016256

In [26]:
from omegaconf import OmegaConf

In [37]:
cfg = OmegaConf.load('base_text_only_task.yaml')

In [38]:
cfg = cfg['model']

In [39]:
cfg_default = Data2VecMultiConfig()

In [40]:
cfg = OmegaConf.merge(cfg_default, cfg)

In [42]:
cfg.keys()

dict_keys(['_name', 'loss_beta', 'loss_scale', 'depth', 'start_drop_path_rate', 'end_drop_path_rate', 'num_heads', 'norm_eps', 'norm_affine', 'encoder_dropout', 'post_mlp_drop', 'attention_dropout', 'activation_dropout', 'dropout_input', 'layerdrop', 'embed_dim', 'mlp_ratio', 'layer_norm_first', 'average_top_k_layers', 'end_of_block_targets', 'clone_batch', 'layer_norm_target_layer', 'batch_norm_target_layer', 'instance_norm_target_layer', 'instance_norm_targets', 'layer_norm_targets', 'ema_decay', 'ema_same_dtype', 'log_norms', 'ema_end_decay', 'ema_anneal_end_step', 'ema_encoder_only', 'max_update', 'modalities', 'shared_decoder', 'min_target_var', 'min_pred_var', 'supported_modality', 'mae_init', 'seed', 'skip_ema', 'cls_loss', 'recon_loss', 'd2v_loss', 'decoder_group'])

In [1]:
from collections import namedtuple
from fairseq.data import Dictionary
import os
Task = namedtuple('Task', 'source_dictionary')

# Create an instance
dictionary = Dictionary.load(os.path.join('../data', "dict.txt"))
dummy_task = Task(source_dictionary=)

In [8]:
d=Dictionary.load(os.path.join('../data', "dict.txt"))
d.add_symbol("<mask>")

50264

In [51]:
dummy_task.source_dictionary

In [52]:
model = Data2VecMultiModel.build_model(cfg, task=dummy_task)

In [53]:
model.load_state_dict(state_dict['model'])

RuntimeError: Error(s) in loading state_dict for Data2VecMultiModel:
	size mismatch for modality_encoders.TEXT.local_encoder.embed_tokens.weight: copying a param with shape torch.Size([50265, 768]) from checkpoint, the shape in current model is torch.Size([50264, 768]).